In [1]:
# Package Import Section
import numpy as np
import matplotlib.pyplot as plt
import numexpr as ne
import timeit
import astropy.units as u
from scipy.special import erfc
from numba import jit

# Specifying the Gaussian Beamlet
Any Gaussian beam is specified by 3 quantities: the rayleigh range, beam waist, and wavelength. If two of these parameters are known, then the beam waist radius, radius of curvature, and the third parameter can be derived. (From Goodman).

$$ \omega_{o} = \sqrt{\frac{\lambda z_{o}}{\pi}} $$
$$ \omega(z) = \omega_{o}\sqrt{1 + (\frac{z}{z_o})^{2}} $$
$$ R(z) = z*(1+(\frac{z_o}{z})^{2}) $$

To decompose a wavefront with a sum of beamlets parameterized by the quantities above, we must determine the overlap factor of the beamlets. This is defined as the  width of a beamlet divided by the separation of an adjacent beamlet.

$$ OF = \frac{2\omega_o}{C_b} $$

A lower overlap factor leads to ripple in the decomposed wavefront, but a high overlap factor leads to increased computational burden. FRED is a commercial software by Photon engineering that suggests an overlap factor of OF = 1.7

In [2]:
# Beamlet Parameters
wl = 2.2e-6 * u.meter # beamlet wavelength
OF = 1.7 # Overlap Factor
wo = 3*wl # beamlet waist
zr = np.pi*wo**2/wl

# Calculating the Amplitude Coefficients (Figure out Implementation)
To arbitrarilly represent a wavefront, the amplitudes of the gaussian beamlets must be calculated. For a set of N gaussian beams, the amplitude coefficients $c_k$ of individual beams should be determined such that the total amplitude $a(x_i)$ at a given position is given by

$$ a(x_i) = \sum^{N}_{k = 1} c_k g_{k}(x_i) $$

For a set of M points in space, this can be written in a matrix vector form

$$
G = 
\begin{bmatrix}
    g_{11} & g_{12} & ... & g_{1N} \\
    g_{21} & g_{22} & ... & g_{2N} \\
    \vdots & \vdots & ... & \vdots \\
    g_{M1} & g_{M2} & ... & g_{MN} \\
\end{bmatrix}
$$

$$
c = 
\begin{bmatrix}
c_1 \\
c_2 \\
\vdots \\
c_N \\
\end{bmatrix}
$$

$$
a = 
\begin{bmatrix}
a_1 \\
a_2 \\
\vdots \\
a_M \\
\end{bmatrix}
$$

$$\textbf{G*c = a}$$

where $g_{ij}$ is the amplitude of the jth beam at the ith point, $c_j$ is the amplitude coefficient of the jth beam, and $a_i$ is the total amplitude at the ith point. For a large aperture, $G$ is sparse. If $M = N$, then there is a unique solution. However, the typical use case is for when $M > N$ (number of data points exceeds beamlets). There are multiple solutions to this problem, but the best coefficient vector minimizes the mean square error, which is given by:

$$
\textbf{c} = (G^{T}G)^{-1}G^{T}\textbf{a}
$$

The RMS error is given by:
$$
RMS = \sqrt{\frac{\sum^{m}_{i = 1} \Delta a_i^{2}}{M}}
$$

# The Complex Curvature Matrix
The previously defined parameters of a generally astigmatic Gaussian Beamlet are given by the complex curvature:
$$
q^{-1} = \frac{1}{R(z)} - i\frac{\lambda}{n\pi\omega(z)^{2}}
$$
$q^{-1}$ is the inverse of the complex curvature, and n is the refractive index of the medium.

The case for a generally astigmatic gaussian beamlet is given by a 2x2 matrix

$$
Q^{-1} =
\begin{bmatrix}
q^{-1}_{xx} & q^{-1}_{xy} \\
q^{-1}_{yx} & q^{-1}_{yy} \\
\end{bmatrix}
$$

$q^{-1}_{xx}$ and $q^{-1}_{yy}$ describe the complex curvature in the x & y directions, whereas $q^{-1}_{xy}$ and $q^{-1}_{yx}$ describe the coupling of the x curvature into the y curvature, and vice-versa. For the orthogonal gaussian beam that hasn't been rotated, the complex curvature matrix looks like:

$$
Q^{-1} =
\begin{bmatrix}
q^{-1} & 0 \\
0 & q^{-1} \\
\end{bmatrix}
$$

With the same curvature in x & y, and no cross-coupling. The coupling coefficients come into play after a rotation matrix is applied to the system, or to an individual beamlet. This will become relevant when truncated beamlets are investigated.

## The gaussian beam
A TEM0,0 gaussian beam can be represented by this complex curvature matrix, given that it encodes the radius of curvature & beam waist size.

$$
E = E_o exp(\frac{-ik}{2} r^{T} Q^{-1} r)
$$

Where $r = [x,y]^{T}$ is the position vector of the gaussian beam, k is the wavenumber, and Q is the complex curvature matrix. The expression for the electric field can also be given by simply multiplying out the exponential

$$
E = E_o exp(\frac{-ik}{2}(q^{-1}_{xx}x^{2} + (q^{-1}_{xy} + q^{-1}_{yx})xy + q^{-1}_{yy}y^{2})
$$

In [3]:
# Define a Q Matrix - diagonal zero for nonastigmatic, nonrotated case
qxx = 1/(1j*zr)
qxy = 0
qyx = 0
qyy = 1/(1j*zr)
Qm = np.array([[qxx,qxy],
             [qyx,qyy]],dtype='complex') # Defines the matrix of inverse q parameters

# Building a System
## Defining the number of beamlets to trace

For a rectangular grid sampling, Worku and Gross (2018) defines the number of beamlets across one dimension (W) of the rectangle as:

$$
N_g = \frac{W * OF}{2\omega_o}
$$

We will mostly be dealing with circular apertures in optics, so this definition does not quite suffice. Using the same expression for the number of beamlets across the radius $N_{beams} = \frac{N_g}{2}$. The total number of beamlets to trace would then be:

$$
N = Round(\pi * N_g^{2})
$$

## Defining our detector

Naturally these beamlets must ultimately be captured by some detector. These are typically square, so we will define how many pixels are across a dimension (npix), as well as the dimension itself (dimd).

## ABCD Raytracing

ABCD raytracing is typically done with a rotationally symmetric system, but we want to account for the generally astigmatic case so we replace the typical 2x2 matries with 4x4 matrices accounting for projections in the x and y direction. First, we define a base ray:

$$
\begin{bmatrix}
x \\
y \\
x' \\
y' \\
\end{bmatrix}
$$

x & y correspond to the position of the base ray in the plane transverse to propagation, x' and y' are the slopes of the base ray in these directions. To define the free space propagation matrix:

$$
T =
\begin{bmatrix}
1 & 0 & \frac{t}{n} & 0 \\
0 & 1 & 0 & \frac{t}{n} \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 \\
\end{bmatrix}
$$

And the Refraction matrix (for systems of power = $\phi$)
$$
R =
\begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
-\phi & 0 & 1 & 0 \\
0 & -\phi & 0 & 1 \\
\end{bmatrix}
$$

These matrices are presented for the orthogonal case, but can be rotated, as with the complex curvature matrix


In [4]:
# Create system
W = 5e-3 * u.meter
npix = 512
dimd = 1e-2 * u.meter

# Number of Beamlets
N = 50 #int(np.floor(np.pi*(W*OF/(2*wo)))*4)
print(N)

50


In [5]:
# Create List of Positions (X,Y) in a Fibbonacci Sampled Spiral
c = np.array([0,0]) # XY offset from a spiral
R = W*np.sqrt(np.linspace(1/2,N-1/2,N))/np.sqrt(N-1/2)
T = 4/(1+np.sqrt(5))*np.pi*np.linspace(1,N,N);
X = c[0] +R*np.cos(T)
Y = c[1] +R*np.sin(T)

In [6]:
# Create the Base Ray
base = np.array([X,
                Y,
                0*X,
                0*Y]) # slopes are all 0 for the base ray

In [7]:
# Package Import Section
import numpy as np
import matplotlib.pyplot as plt
import numexpr as ne
import timeit
import astropy.units as u
from scipy.special import erfc
from numba import jit

# Creating an Optical System Class To propagate rays
class GaubletOpticalSystem:
    
    def __init__(self,
                 epd,
                 npix,
                 dimd,
                 wavelength,
                 numbeamlets):
        
        basesys = np.array([[1,0,0,0],
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]])
        self.system = basesys
        self.epd = epd
        self.npix = npix
        self.dimd = dimd
        
        # Beamlet Parameters
        self.wl = wavelength * u.meter # beamlet wavelength
        OF = 2 # Overlap Factor
        wo = 3*self.wl # beamlet waist
        zr = np.pi*wo**2/self.wl
        
        # Define a Q Matrix - diagonal zero for nonastigmatic, nonrotated case
        qxx = 1/(1j*zr)
        qxy = 0
        qyx = 0
        qyy = 1/(1j*zr)
        self.Q = np.array([[qxx,qxy],
                            [qyx,qyy]],dtype='complex') # Defines the matrix of inverse q parameters
        
        # Create system with a circular aperture for testing - arbitrary for now
        W = 5e-3 * u.meter
        npix = 512
        dimd = 1e-2 * u.meter

        # Number of Beamlets
        self.N = numbeamlets #int(np.floor(np.pi*(W*OF/(2*wo)))*4)
        
        # Create List of Positions (X,Y) in a Fibbonacci Sampled Spiral
        c = np.array([0,0]) # XY offset from a spiral
        R = W*np.sqrt(np.linspace(1/2,self.N-1/2,self.N))/np.sqrt(self.N-1/2)
        T = 4/(1+np.sqrt(5))*np.pi*np.linspace(1,self.N,self.N);
        X = c[0] +R*np.cos(T)
        Y = c[1] +R*np.sin(T)
        
        # Create the Base Rays
        self.baserays = np.array([X,
                                  Y,
                                  0*X,
                                  0*Y]) # slopes are all 0 for the base ray
        
        
        

        
    def add_optic(self,efl):
        efl = efl
        
        # Focusing matrix
        optic = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [-1/efl,0,1,0],
                  [0,-1/efl,0,1]])
        self.system = np.matmul(optic,self.system)
        
    def add_distance(self,distance,index):
        distance = distance
        index = index
        
        # Propagation matrix
        propg = np.array([[1,0,distance/index,0],
                          [0,1,0,distance/index],
                          [0,0,1,0],
                          [0,0,0,1]])
        self.system = np.matmul(propg,self.system)
        
    def propagate(self):
        
        # Propagate the base rays
        prop = np.matmul(self.system,self.baserays)
        
        # Propagate the Q matrix
        A = self.system[0:2,0:2]
        B = self.system[0:2,2:4]
        C = self.system[2:4,0:2]
        D = self.system[2:4,2:4]
        Qprop_n = (C + np.matmul(D,self.Q))
        Qprop_d = np.linalg.inv(A+np.matmul(B,self.Q))
        Qprop   = np.matmul(Qprop_n,Qprop_d)
        return Qprop,prop
    
    #def display():
    
     
class GaubletWavefront:
    
    def __init__(self,
                 wavelength,
                 numbeamlets,
                 npix,
                 dimension,
                 proprays,
                 baserays,
                 Qorig,
                 Qprop,
                 system):
        
        self.wavelength = wavelength
        self.numbeamlets = numbeamlets
        self.npix = npix
        self.dimension = dimension
        self.proprays = proprays
        self.baserays = baserays
        self.Q = Qorig
        self.Qprop = Qprop
        self.system = system
        u = np.linspace(-self.dimension,self.dimension,self.npix)
        v = np.linspace(-self.dimension,self.dimension,self.npix)
        self.u,self.v = np.meshgrid(u,v)
        
        # pre-define a datacube to dump the Gaublet phase in
        self.Dphase = np.zeros([npix,npix,self.numbeamlets],dtype='complex')
    
    def Phasecalc(self): # returns datacube of gaublet phases
        
        # propagation distance for a beamlet
        lo = np.sqrt((self.proprays[0,:] - self.baserays[0,:])**2 + (self.proprays[1,:] - self.baserays[1,:])**2 + self.system[0,2]**2) 
        A  = self.system[0:2,0:2]
        B  = self.system[0:2,2:4]
        phase = self.Phasecube(self.numbeamlets,
                               self.dimension,
                               self.npix,
                               self.proprays,
                               self.wavelength,
                               self.Qprop,
                               lo,
                               self.Dphase,
                               self.u,
                               self.v)
        phasor = ne.evaluate('exp(phase)')
        Ephase = np.sum(phasor,axis=2)/np.sqrt(np.linalg.norm(A+np.matmul(B,self.Q)))
        
        return Ephase
    
    @staticmethod
    @jit(nopython=True)
    def Phasecube(numbeamlets,dimension,npix,proprays,wavelength,Qprop,lo,Dphase,u,v):
        for ind in range(numbeamlets):
        
            up = u - proprays[0,ind]
            vp = v - proprays[1,ind]
            
            tran_phase = ((-1j*(2*np.pi/wavelength)/2)*(Qprop[0,0]*up**2 + (Qprop[1,0] + Qprop[0,1])*up*vp + Qprop[1,1]*vp**2))
            long_phase = (-1j*(2*np.pi/wavelength)*lo[ind])
            Dphase[:,:,ind] = tran_phase+long_phase
            
        return Dphase
    
    def display(self,phase):
        
        self.phase = phase
        
        # displays field amplitude, phase, and irradiance
        u = np.linspace(-self.dimension,self.dimension,self.npix)
        v = np.linspace(-self.dimension,self.dimension,self.npix)
        
        plt.figure(figsize=[17,9])
        plt.subplot(121)
        plt.set_cmap('gray')
        plt.pcolor(u,v,np.real(np.sqrt(self.phase*np.conj(self.phase))))
        plt.title('Field Amplitude')
        plt.xlabel('Detector Dimension [m]')
        plt.ylabel('Detector Dimension [m]')
        
        plt.subplot(122)
        plt.pcolor(u,v,np.angle((self.phase)))
        plt.title('Field Phase')
        plt.xlabel('Detector Dimension [m]')
        plt.ylabel('Detector Dimension [m]')

    

In [8]:
osys = GaubletOpticalSystem(epd=5e-3,npix=1024,dimd=5e-3,wavelength=2.2e-6,numbeamlets=1000)
osys.add_optic(efl=5e-3)
osys.add_distance(distance=5e-3,index=1)
Qp,prop = osys.propagate()

# fun parameters
# epd = 5e-3
# npix = 512
# dimd = 1e-2
# wl = 2.2e-6
# numbeamlets = 3000

In [ ]:
gwfr = GaubletWavefront(wavelength=osys.wl,numbeamlets=osys.N,npix=osys.npix,dimension=osys.dimd,proprays=prop,baserays=osys.baserays,Qorig=osys.Q,Qprop=Qp,system = osys.system)
Dphase = gwfr.Phasecalc()

In [ ]:
gwfr.display(phase=Dphase)


In [ ]:
print(osys.N)

In [ ]:
# Package Import Section
import numpy as np
import matplotlib.pyplot as plt
import numexpr as ne
import timeit
import astropy.units as u
from scipy.special import erfc
from numba import jit

# Creating an Optical System Class To propagate rays
class GaubletOpticalSystem:

    def __init__(self,
                 epd,
                 npix,
                 dimd,
                 wavelength,
                 numbeamlets):

        basesys = np.array([[1,0,0,0],
                            [0,1,0,0],
                            [0,0,1,0],
                            [0,0,0,1]])
        self.system = basesys
        self.epd = epd
        self.npix = npix
        self.dimd = dimd# Number of Beamlets
        self.N = numbeamlets #int(np.floor(np.pi*(W*OF/(2*wo)))*4)

        # THIS BLOCK IS HARD-CODED, CHANGE FOR FINAL VERSION ##################################################

        # Beamlet Parameters
        self.wl = wavelength * u.meter # beamlet wavelength
        OF = 2 # Overlap Factor
        wo = 10*self.wl # beamlet waist
        zr = np.pi*wo**2/self.wl

        # Create system with a circular aperture for testing - arbitrary for now
        W = 5e-3 * u.meter
        npix = 512
        dimd = 1e-2 * u.meter

        # Create List of Positions (X,Y) in a Fibbonacci Sampled Spiral Circular Aperture
        c = np.array([0,0]) # XY offset from a spiral
        R = W*np.sqrt(np.linspace(1/2,self.N-1/2,self.N))/np.sqrt(self.N-1/2)
        T = 4/(1+np.sqrt(5))*np.pi*np.linspace(1,self.N,self.N);
        X = c[0] +R*np.cos(T)
        Y = c[1] +R*np.sin(T)

        # THIS BLOCK IS HARD-CODED, CHANGE FOR FINAL VERSION ##################################################


        # Define a Q Matrix - diagonal zero for nonastigmatic, nonrotated case
        qxx = 1/(1j*zr)
        qxy = 0
        qyx = 0
        qyy = 1/(1j*zr)
        self.Q = np.array([[qxx,qxy],
                            [qyx,qyy]],dtype='complex') # Defines the matrix of inverse q parameters

        # Create the Base Rays
        self.baserays = np.array([X,
                                  Y,
                                  0*X,
                                  0*Y]) # slopes are all 0 for the base ray

    def add_optic(self,efl):
        efl = efl

        # Focusing matrix
        optic = np.array([[1,0,0,0],
                  [0,1,0,0],
                  [-1/efl,0,1,0],
                  [0,-1/efl,0,1]])
        self.system = np.matmul(optic,self.system)

    def add_distance(self,distance,index):
        distance = distance
        index = index

        # Propagation matrix
        propg = np.array([[1,0,distance/index,0],
                          [0,1,0,distance/index],
                          [0,0,1,0],
                          [0,0,0,1]])
        self.system = np.matmul(propg,self.system)

    def propagate(self):

        # Propagate the base rays
        prop = np.matmul(self.system,self.baserays)

        # Propagate the Q matrix
        A = self.system[0:2,0:2]
        B = self.system[0:2,2:4]
        C = self.system[2:4,0:2]
        D = self.system[2:4,2:4]
        Qprop_n = (C + np.matmul(D,self.Q))
        Qprop_d = np.linalg.inv(A+np.matmul(B,self.Q))
        Qprop   = np.matmul(Qprop_n,Qprop_d)
        return Qprop,prop

    #def display():


class GaubletWavefront:

    def __init__(self,
                 wavelength,
                 numbeamlets,
                 npix,
                 dimension,
                 proprays,
                 baserays,
                 Qorig,
                 Qprop,
                 system):

        self.wavelength = wavelength
        self.numbeamlets = numbeamlets
        self.npix = npix
        self.dimension = dimension
        self.proprays = proprays
        self.baserays = baserays
        self.Q = Qorig
        self.Qprop = Qprop
        self.system = system
        u = np.linspace(-self.dimension,self.dimension,self.npix)
        v = np.linspace(-self.dimension,self.dimension,self.npix)
        self.u,self.v = np.meshgrid(u,v)

        # pre-define a datacube to dump the Gaublet phase in
        self.Dphase = np.zeros([npix,npix,self.numbeamlets],dtype='complex')

    def Phasecalc(self): # returns datacube of gaublet phases

        # propagation distance for a beamlet
        lo = np.sqrt((self.proprays[0,:] - self.baserays[0,:])**2 + (self.proprays[1,:] - self.baserays[1,:])**2 + self.system[0,2]**2)
        A  = self.system[0:2,0:2]
        B  = self.system[0:2,2:4]
        phase = self.Phasecube(self.numbeamlets,
                               self.dimension,
                               self.npix,
                               self.proprays,
                               self.wavelength,
                               self.Qprop,
                               lo,
                               self.Dphase,
                               self.u,
                               self.v)
        phasor = ne.evaluate('exp(phase)')
        Ephase = np.sum(phasor,axis=2)/np.sqrt(np.linalg.norm(A+np.matmul(B,self.Q)))

        return Ephase

    @staticmethod
    @jit(nopython=True)
    def Phasecube(numbeamlets,dimension,npix,proprays,wavelength,Qprop,lo,Dphase,u,v):
        for ind in range(numbeamlets):

            up = u - proprays[0,ind]
            vp = v - proprays[1,ind]

            tran_phase = ((-1j*(2*np.pi/wavelength)/2)*(Qprop[0,0]*up**2 + (Qprop[1,0] + Qprop[0,1])*up*vp + Qprop[1,1]*vp**2))
            long_phase = (-1j*(2*np.pi/wavelength)*lo[ind])
            Dphase[:,:,ind] = tran_phase+long_phase

        return Dphase

    def display(self,phase):

        self.phase = phase

        # displays field amplitude, phase, and irradiance
        u = np.linspace(-self.dimension,self.dimension,self.npix)
        v = np.linspace(-self.dimension,self.dimension,self.npix)

        plt.figure(figsize=[17,9])
        plt.subplot(121)
        plt.set_cmap('gray')
        plt.pcolor(u,v,np.real(np.sqrt(self.phase*np.conj(self.phase))))
        plt.title('Field Amplitude')
        plt.xlabel('Detector Dimension [m]')
        plt.ylabel('Detector Dimension [m]')

        plt.subplot(122)
        plt.pcolor(u,v,np.angle((self.phase)))
        plt.title('Field Phase')
        plt.xlabel('Detector Dimension [m]')
        plt.ylabel('Detector Dimension [m]')






In [ ]:
# Test System
osys = GaubletOpticalSystem(epd=5e-3,npix=256,dimd=1e-2,wavelength=2.2e-6,numbeamlets=3000)
osys.add_optic(efl=5e100)
osys.add_distance(distance=5,index=1)
Qp,prop = osys.propagate()
gwfr = GaubletWavefront(wavelength=osys.wl,numbeamlets=osys.N,npix=osys.npix,dimension=osys.dimd,proprays=prop,baserays=osys.baserays,Qorig=osys.Q,Qprop=Qp,system = osys.system)
Dphase = gwfr.Phasecalc()
gwfr.display(phase=Dphase)


# fun parameters
# epd = 5e-3
# npix = 512
# dimd = 1e-2
# wl = 2.2e-6
# numbeamlets = 3000